In [1]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE, ADASYN
from collections import Counter
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn import metrics
import os
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
import pydotplus
from sklearn import tree
import collections
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import accuracy_score as acs
import seaborn as sns
import pickle

# Prepare Training Set

In [2]:
df = pd.read_csv("/Users/Tim/Desktop/ML with Python/Assignment23_March6/dataset/train.csv", encoding='utf8')
del df['id']
del df['purchaseTime']
del df['visitTime']
del df['C1']
del df['C2']
del df['C3']
del df['C4']
del df['C5']
del df['C6']
del df['C8']
del df['C10']
del df['C11']
del df['C12']
del df['N1']
del df['N2']


categorical_features = ['hour','C9','C7']
df[categorical_features] = df[categorical_features].astype('category')
df=pd.get_dummies(df) 
del df['C9_2711086415']

In [3]:
X = df.drop(['label'],axis=1)
Y = df['label']

X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,random_state=666,shuffle=True,stratify=Y)

Train=X_train.join(Y_train)
X_train

,N3,N4,N5,N6,N7,N8,N9,N10,hour_0,hour_1,...,C9_3623356211,C9_3738539033,C9_3790028721,C9_3828882129,C9_3863488499,C9_3878556116,C9_3898085784,C9_3911032568,C9_4054959403,C9_4287642944
25728,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26853,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11783,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18248,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27907,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15752,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25708,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1764,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9285,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


# Re-sampling

In [4]:
df = pd.read_csv("/Users/Tim/Desktop/ML with Python/Assignment23_March6/dataset/train.csv", encoding='utf8')
del df['id']
del df['purchaseTime']
del df['visitTime']
del df['C1']
del df['C2']
del df['C3']
del df['C4']
del df['C5']
del df['C6']
del df['C8']
del df['C10']
del df['C11']
del df['C12']
del df['N1']
del df['N2']


categorical_features = ['hour','C9','C7']
df[categorical_features] = df[categorical_features].astype('category')
df=pd.get_dummies(df) 

numerical_features = ['N3','N4','N5','N6','N7','N8','N9','N10']
#Standardize te numerical variables
#from sklearn import preprocessing
#min_max_scaler = preprocessing.MinMaxScaler()
#df[numerical_features] = min_max_scaler.fit_transform(df[numerical_features])

from scipy import stats
z = np.abs(stats.zscore(df[numerical_features]))
df[numerical_features] = df[numerical_features][(z < 20).all(axis=1)]
dft = df
#['visitTime','purchaseTime','C1','C3','C10','N4','N6','N10']

df["label"].value_counts()

from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
dft[numerical_features] = imputer.fit_transform(dft[numerical_features])
del dft['C9_2711086415']

In [5]:
Xt = dft.drop(['label'],axis=1)
Yt = dft['label']

X_traint, X_testt, Y_traint, Y_testt = train_test_split(Xt,Yt,test_size=0.2,random_state=666,shuffle=True,stratify=Y)

Traint=X_traint.join(Y_traint)

In [6]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

X_totrain = Traint.drop(['label'],axis=1)
y_totrain = Traint['label']

# define pipeline
over = SMOTE(sampling_strategy=0.01,random_state=666)
under = RandomUnderSampler(sampling_strategy=0.01,random_state=666)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)
# transform the dataset
X_totrain, y_totrain = pipeline.fit_resample(X_totrain, y_totrain)
toTrain=pd.concat([y_totrain,X_totrain],axis=1,join='inner')
toTrain["label"].value_counts()

-1    25000
 1      250
Name: label, dtype: int64

In [7]:
X_toTrain = toTrain.drop(['label'],axis=1)
Y_toTrain = toTrain['label']

In [8]:
import xgboost as xgb
models = []
#models.append(('XGB',xgb.XGBClassifier(max_depth=10,random_state=666,learning_rate=0.01,n_estimators=2000,min_child_weight=4,n_jobs=-1,colsample_bytree=0.4,
#                                      gamma=10,subsample=0.5,reg_alpha=0.5,objective='binary:logistic',booster='gbtree')))
models.append(('RF', RandomForestClassifier(max_depth=15,random_state=666)))
#clf = xgb.train({'max_depth': 60, 'num_class': n_class, 'gamma': 10,'colsample_bytree': 0.4, 'subsample': 0.7, 'num_round': 10000, 'min_child_weight': 4, 'n_estimators': 2000, 'n_jobs': -1, 'learning_rate': 0.01, 'reg_alpha': 0.5},dTrain, num_boost_round = 200)

#XGBClassifier(max_depth=3, learning_rate=0.1, n_estimators=100, silent=True, objective='binary:logistic', booster='gbtree', n_jobs=1, nthread=None, gamma=0, min_child_weight=1, max_delta_step=0, subsample=1, colsample_bytree=1, colsample_bylevel=1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, base_score=0.5, random_state=0, seed=None, missing=None, **kwargs)

print('TrainingSet')
# evaluate each model in turn
results = []
names = []
for name, model in models:
    model = model.fit(X_toTrain, Y_toTrain)
    trainpred = model.predict(X_train)
    print(metrics.accuracy_score(Y_train,trainpred))
    print(metrics.classification_report(Y_train,trainpred))


print('\n')

print('TestingSet')

for name, model in models:
    y_pred = model.predict(X_test)
#Evaluate predictions
    print(metrics.accuracy_score(Y_test,y_pred))
#print(metrics.confusion_matrix(Y_validation,y_pred))
    print(metrics.classification_report(Y_test,y_pred))

TrainingSet
0.9995227110015115
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00     25096
           1       0.90      0.83      0.86        46

    accuracy                           1.00     25142
   macro avg       0.95      0.91      0.93     25142
weighted avg       1.00      1.00      1.00     25142



TestingSet
0.999522748965956
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00      6275
           1       0.90      0.82      0.86        11

    accuracy                           1.00      6286
   macro avg       0.95      0.91      0.93      6286
weighted avg       1.00      1.00      1.00      6286



In [9]:
feature_imp = pd.Series(model.feature_importances_,index=list(X.columns)).sort_values(ascending=False)

my_list = list(feature_imp[0:20].keys())
my_list

['N9',
 'N6',
 'N10',
 'N8',
 'N3',
 'N7',
 'C7_2303407174',
 'N4',
 'N5',
 'C9_3911032568',
 'C9_3310009198',
 'C9_3598157468',
 'C9_14692648',
 'C7_1342367140',
 'C7_730319316',
 'C9_4287642944',
 'hour_16',
 'hour_7',
 'C9_1421481761',
 'hour_11']

# Save the model & test with the Testing CSV

In [10]:
# save the model to disk
filename = '/Users/Tim/Desktop/ML with Python/Assignment23_March6/dataset/model.sav'
pickle.dump(model, open(filename, 'wb'))
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))

In [11]:
df_1 = df[df['label']==1]

In [12]:
testdf = pd.read_csv("/Users/Tim/Desktop/ML with Python/Assignment23_March6/dataset/test1.csv", encoding='utf8')

del testdf['id']
del testdf['purchaseTime']
del testdf['visitTime']
del testdf['C1']
del testdf['C2']
del testdf['C3']
del testdf['C4']
del testdf['C5']
del testdf['C6']
del testdf['C8']
del testdf['C10']
del testdf['C11']
del testdf['C12']
del testdf['N1']
del testdf['N2']

categorical_features = ['hour','C9','C7']
testdf[categorical_features] = testdf[categorical_features].astype('category')
testdf=pd.get_dummies(testdf) 
testdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32086 entries, 0 to 32085
Data columns (total 98 columns):
label            32086 non-null int64
N3               32086 non-null int64
N4               32086 non-null int64
N5               32086 non-null int64
N6               32086 non-null int64
N7               32086 non-null int64
N8               32086 non-null int64
N9               32086 non-null int64
N10              32086 non-null int64
hour_0           32086 non-null uint8
hour_1           32086 non-null uint8
hour_2           32086 non-null uint8
hour_3           32086 non-null uint8
hour_4           32086 non-null uint8
hour_5           32086 non-null uint8
hour_6           32086 non-null uint8
hour_7           32086 non-null uint8
hour_8           32086 non-null uint8
hour_9           32086 non-null uint8
hour_10          32086 non-null uint8
hour_11          32086 non-null uint8
hour_12          32086 non-null uint8
hour_13          32086 non-null uint8
hour_14          

In [13]:
#listtrain = Traint.columns
#listtest = testdf.columns 
#ret=list(set(listtrain).difference(set(listtest)))

In [14]:
testdf=pd.concat([testdf,df_1])

In [15]:
Xtest = testdf.drop(['label'],axis=1)
Ytest = testdf['label']
Xtest

,N3,N4,N5,N6,N7,N8,N9,N10,hour_0,hour_1,...,C9_3623356211,C9_3738539033,C9_3790028721,C9_3828882129,C9_3863488499,C9_3878556116,C9_3898085784,C9_3911032568,C9_4054959403,C9_4287642944
0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29362,0.157967,0.112396,0.000924,0.011568,0.00086,0.344391,0.020778,0.030274,0,0,...,0,0,0,0,0,0,0,0,0,1
29504,0.157967,0.112396,0.000924,0.011568,0.00086,0.344391,0.020778,0.030274,0,0,...,0,0,0,0,0,0,0,0,0,1
29715,0.157967,0.112396,0.000924,0.011568,0.00086,0.344391,0.020778,0.030274,0,0,...,0,0,0,0,0,0,0,0,0,1
30219,0.157967,0.112396,0.000924,0.011568,0.00086,0.344391,0.020778,0.030274,0,0,...,0,0,0,0,0,0,0,0,0,1


In [16]:
model = loaded_model.fit(X_toTrain, Y_toTrain)
y_testpred = model.predict(Xtest)
print(metrics.accuracy_score(Ytest,y_testpred))
#print(metrics.confusion_matrix(Y_validation,y_pred))
print(metrics.classification_report(Ytest,y_testpred))

0.9995644463802383
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00     32086
           1       0.94      0.81      0.87        57

    accuracy                           1.00     32143
   macro avg       0.97      0.90      0.93     32143
weighted avg       1.00      1.00      1.00     32143



In [17]:
testdf

,label,N3,N4,N5,N6,N7,N8,N9,N10,hour_0,...,C9_3623356211,C9_3738539033,C9_3790028721,C9_3828882129,C9_3863488499,C9_3878556116,C9_3898085784,C9_3911032568,C9_4054959403,C9_4287642944
0,-1,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1,...,0,0,0,0,0,0,0,0,0,0
1,-1,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1,...,0,0,0,0,0,0,0,0,0,0
2,-1,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1,...,0,0,0,0,0,0,0,0,0,0
3,-1,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1,...,0,0,0,0,0,0,0,0,0,0
4,-1,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29362,1,0.157967,0.112396,0.000924,0.011568,0.00086,0.344391,0.020778,0.030274,0,...,0,0,0,0,0,0,0,0,0,1
29504,1,0.157967,0.112396,0.000924,0.011568,0.00086,0.344391,0.020778,0.030274,0,...,0,0,0,0,0,0,0,0,0,1
29715,1,0.157967,0.112396,0.000924,0.011568,0.00086,0.344391,0.020778,0.030274,0,...,0,0,0,0,0,0,0,0,0,1
30219,1,0.157967,0.112396,0.000924,0.011568,0.00086,0.344391,0.020778,0.030274,0,...,0,0,0,0,0,0,0,0,0,1


In [18]:
loaded_model.predict_proba(Xtest)

array([[9.99549762e-01, 4.50238363e-04],
       [9.99788855e-01, 2.11144786e-04],
       [9.99788855e-01, 2.11144786e-04],
       ...,
       [8.16163837e-02, 9.18383616e-01],
       [8.16163837e-02, 9.18383616e-01],
       [9.81706894e-01, 1.82931062e-02]])

In [19]:
testdf['prob0'] = loaded_model.predict_proba(Xtest)[0:,0]

In [20]:
testdf['prob1'] = loaded_model.predict_proba(Xtest)[0:,1]

In [21]:
testdf['predtict'] = y_testpred
testdf

,label,N3,N4,N5,N6,N7,N8,N9,N10,hour_0,...,C9_3828882129,C9_3863488499,C9_3878556116,C9_3898085784,C9_3911032568,C9_4054959403,C9_4287642944,prob0,prob1,predtict
0,-1,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1,...,0,0,0,0,0,0,0,0.999550,0.000450,-1
1,-1,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1,...,0,0,0,0,0,0,0,0.999789,0.000211,-1
2,-1,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1,...,0,0,0,0,0,0,0,0.999789,0.000211,-1
3,-1,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1,...,0,0,0,0,0,0,0,0.999921,0.000079,-1
4,-1,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1,...,0,0,0,0,0,0,0,0.999550,0.000450,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29362,1,0.157967,0.112396,0.000924,0.011568,0.00086,0.344391,0.020778,0.030274,0,...,0,0,0,0,0,0,1,0.119250,0.880750,1
29504,1,0.157967,0.112396,0.000924,0.011568,0.00086,0.344391,0.020778,0.030274,0,...,0,0,0,0,0,0,1,0.135119,0.864881,1
29715,1,0.157967,0.112396,0.000924,0.011568,0.00086,0.344391,0.020778,0.030274,0,...,0,0,0,0,0,0,1,0.081616,0.918384,1
30219,1,0.157967,0.112396,0.000924,0.011568,0.00086,0.344391,0.020778,0.030274,0,...,0,0,0,0,0,0,1,0.081616,0.918384,1


In [22]:
#Finalize 
x=testdf[testdf['label']== -1]['prob0']

In [23]:
finalresult = pd.read_csv("/Users/Tim/Desktop/ML with Python/Assignment23_March6/dataset/test1.csv", encoding='utf8')

In [24]:
finalresult=pd.concat([finalresult['id'],x],axis=1)

In [25]:
finalresult

,id,prob0
0,1,0.999550
1,2,0.999789
2,3,0.999789
3,4,0.999921
4,5,0.999550
...,...,...
32081,32082,0.988728
32082,32083,0.999487
32083,32084,0.999487
32084,32085,0.988728


In [26]:
finalresult.to_csv("/Users/Tim/Desktop/ML with Python/Assignment23_March6/prediction_e197472_DingYao.csv",index=False)